In [ ]:
import pandas as pd

In [ ]:
!unzip '/content/train.csv.zip'

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df_new = df.copy()

In [ ]:
df_new = df_new.sample(n=50000)

In [ ]:
df_new = df_new.drop(columns = ['id','qid1','qid2'])

In [ ]:
df_new

,question1,question2,is_duplicate
258381,What is molding?,Is this mold?,0
379450,How Do I Runaway From Home?,What are some safe ways to runaway from home?,1
379843,Does Donald Trump have a limited vocabulary?,Why does Donald J. Trump appear to have a limi...,1
158090,What is a verified profile on Quora?,How do you get your profile verified on Quora?,1
278667,What is meant by black hole?,What is called a black hole?,1
...,...,...,...
356129,"When a man wears an ear ring in his right ear,...",What does a black mole near right ear of a man...,0
329769,What is the going rate of pay for freelance wr...,I'm a freelance content writer. Should I do an...,0
352387,What is the basic difference between specific ...,What is basic difference between specific yiel...,1
103976,How can I create an Instagram story using mult...,Is there a limit on the number of photos an ap...,0


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Concatenate, Dot, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import logging
from transformers import logging as transformers_logging

logging.basicConfig(level=logging.CRITICAL)
transformers_logging.set_verbosity(logging.CRITICAL)

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Concatenate, Dropout
from sklearn.model_selection import train_test_split

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
bert_model.trainable = False

X = df_new[['question1', 'question2']]
y = df_new['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Assuming 'Quora' is your DataFrame and 'question1' and 'question2' are the columns
questions1 = X_train['question1'].astype(str)
questions2 = X_train['question2'].astype(str)

# Lists to store individual question pair input ids and masks
input_ids_1 = []
attention_masks_1 = []
input_ids_2 = []
attention_masks_2 = []

# Loop through each pair of questions
for i, (q1, q2) in enumerate(zip(questions1, questions2)):

    encoding_1 = tokenizer.encode_plus(
        q1,
        q2,
        add_special_tokens=True,
        max_length=50,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )

    encoding_2 = tokenizer.encode_plus(
        q2,
        q1,
        add_special_tokens=True,
        max_length=50,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )

    # Append input ids and attention masks to respective lists
    input_ids_1.append(encoding_1["input_ids"])
    attention_masks_1.append(encoding_1["attention_mask"])

    input_ids_2.append(encoding_2["input_ids"])
    attention_masks_2.append(encoding_2["attention_mask"])

# Convert the lists to numpy arrays
input_ids_1 = np.array(input_ids_1)
attention_masks_1 = np.array(attention_masks_1)
input_ids_2 = np.array(input_ids_2)
attention_masks_2 = np.array(attention_masks_2)
questions1_val = X_test['question1'].astype(str)
questions2_val = X_test['question2'].astype(str)

input_ids_1_test = []
attention_masks_1_test = []
input_ids_2_test = []
attention_masks_2_test = []
for i, (q1, q2) in enumerate(zip(questions1_val, questions2_val)):

    encoding_1 = tokenizer.encode_plus(
        q1,
        q2,
        add_special_tokens=True,
        max_length=80,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )
    encoding_2 = tokenizer.encode_plus(
        q2,
        q1,
        add_special_tokens=True,
        max_length=50,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )
    input_ids_1_test.append(encoding_1["input_ids"])
    attention_masks_1_test.append(encoding_1["attention_mask"])
    input_ids_2_test.append(encoding_2["input_ids"])
    attention_masks_2_test.append(encoding_2["attention_mask"])

input_ids_1_test = np.array(input_ids_1_test)
attention_masks_1_test = np.array(attention_masks_1_test)
input_ids_2_test = np.array(input_ids_2_test)
attention_masks_2_test = np.array(attention_masks_2_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,DataCollatorWithPadding,TFAutoModel,DistilBertConfig,TFDistilBertModel, BertConfig, TFBertModel, TFRobertaModel

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


except ValueError:
    tpu_strategy = tf.distribute.get_strategy()
class L1Dist(Layer):

    def __init__(self,**kwargs):
        super().__init__()

    def call(self,embedding1,embedding2):
        return tf.math.abs(embedding1 - embedding2)

with tpu_strategy.scope():
    transformer_model = TFBertModel.from_pretrained("bert-base-uncased")

    input_ids_in1 = Input(shape=(None,),name='input_ids1', dtype='int32')
    input_masks_in1 = Input(shape=(None,), name='attention_mask1', dtype='int32')
    input_ids_in2 = Input(shape=(None,),name='input_ids2', dtype='int32')
    input_masks_in2 = Input(shape=(None,), name='attention_mask2', dtype='int32')

    embedding_layer1 = transformer_model(input_ids_in1, attention_mask=input_masks_in1).last_hidden_state
    embedding_layer2 = transformer_model(input_ids_in2, attention_mask=input_masks_in2).last_hidden_state

    embedding1 = GlobalAveragePooling1D()(embedding_layer1)
    embedding2 = GlobalAveragePooling1D()(embedding_layer2)
    l1_dist = L1Dist()(embedding1,embedding2)
    x = Dense(512, activation='relu')(l1_dist)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_ids_in1, input_masks_in1, input_ids_in2, input_masks_in2], outputs = output)
    model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics='accuracy')


for layer in model.layers[:5]:
    layer.trainable = False

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.3,
                                            min_lr=0.00000001)

In [ ]:
# Assuming labels is defined as before
labels = y_train.astype(int)

label_test = y_test.astype(int)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.3,
                                            min_lr=0.00000001)
history = model.fit(
    x=[
        input_ids_1[:, 0],
        attention_masks_1[:, 0],
        input_ids_2[:, 0],
        attention_masks_2[:, 0]
    ],
    y=labels,
    epochs=5,
    batch_size=32,
    validation_data=(
        (
            np.asarray(input_ids_1_test[:, 0]),
            np.asarray(attention_masks_1_test[:, 0]),
            np.asarray(input_ids_2_test[:, 0]),
            np.asarray(attention_masks_2_test[:, 0]))
        ,
        label_test  # Assuming you have a separate array for validation labels
    ),callbacks =[earlystopping, learning_rate_reduction]
)

Epoch 1/5


1250/1250 [==============================] - 978s 746ms/step - loss: 0.5121 - accuracy: 0.6711 - val_loss: 0.4343 - val_accuracy: 0.8009 - lr: 1.0000e-05
Epoch 2/5
1250/1250 [==============================] - 923s 738ms/step - loss: 0.3874 - accuracy: 0.8309 - val_loss: 0.3893 - val_accuracy: 0.8328 - lr: 1.0000e-05
Epoch 3/5
1250/1250 [==============================] - 922s 738ms/step - loss: 0.3140 - accuracy: 0.8700 - val_loss: 0.3745 - val_accuracy: 0.8376 - lr: 1.0000e-05
Epoch 4/5
1250/1250 [==============================] - 923s 738ms/step - loss: 0.2616 - accuracy: 0.8959 - val_loss: 0.3737 - val_accuracy: 0.8511 - lr: 1.0000e-05
Epoch 5/5
1250/1250 [==============================] - 922s 738ms/step - loss: 0.2124 - accuracy: 0.9208 - val_loss: 0.3649 - val_accuracy: 0.8526 - lr: 1.0000e-05
